# Analyzing patient data (extended version)

## Preliminaries

Load the required modules.

## Data set description

The data consists of two spreadsheets.

The first is data on patients gathered during an experimental trial.  The patients' temperature is measured at one-hour intervals, and each hour, the patients receive a dose of a drug to reduce fever.  The first column is a patient ID, the second is the dose, the third the datetime information, and the fourth the body temperature.

The second spreadsheet is metadata on the patients.  The first column is the patient ID, which should be consistent with that in the first spreadsheet, the second column denotes the gender of the patient, the third an experimental condition.

## Experiment data

Read the data set as a pandas `DataFrame` from an Excel spreadsheet.

Let's explore the data set a bit.

The data contains 4 columns, and 62 rows, and there seems to be missing data for `dose` and `temperature` since these columns have only 61 entries.

Let's check the first few and last few rows.

A very rough statistical overview is also useful.

It seems that we have 9 patients, but lets verify that.

Let's check whether the number of measurements is equal for all 9 patients.

For all patients but one there are 7 measurements, for one there are only six.  Let's compare the timestampts for two patients.

It is quite clear we will have to deal with missing data.

## Analyzing experiment data as a time series

We can now rearrange the data for a time series analysis by using the date as an index, and show the patient data as individual columns.

Note that the missing data for patient 6 has been represented as `NaN`.

Hm, something seems fishy with the data for patient 3, let's visualize it individually.

The temperature for patient 3 was not measured at 13:00:00, but the dose was recorded. Let's check for other missing data more systematically.

We should take a look at the measurements for 13:00:00 and 14:00:00. First for `13:00:00`:

This was the temperature we were aware of, but at 14:00:00, it seems the dose was not recorded for patient 4.

From the plots, it seems reasonable to do a linear interpolation for both missing data points.

Let's do the interpolation on the time series in place.

We should have no more missing data, let's verify.

Let's verify what happend for patient 6 with the missing data at the end of the experiment.

The value is the same for 16h as for 15h, which is reasonable.

## Derived data

Let's add a new column to the time series that represents the average temperature over all patients.

Let's also add a column for each patient that shows the cummulative dose for that patient.

Let's check the maximum temperature and the total dose per patient using a pivot table. The index is the patient ID, and the values the dose and temperature, aggregated as sum and maximum respectively.

Visualized as a scatter plot:

It seems possible that there is a linear relationship between the maximum temperature, and the total dose administered during the experiment.

## Metadata

There is a second file with metadata on the patients, let's read that as well. The file is `data/patient_metadata.xlsx`.


How many distinct genders and conditions do we have?

Gender and condition are categorical data, let's use `describe` on those columns.

Which patients are male, and have condition A?

## Merging dataframes

The patient IDs in the exerperimental and metadata should correspond, so let merge the dataframes. However, let's first check whether the patient IDs are the same in both dataframes.

So metadata is missing for patient 4, and there is metadata for patients that were not involved in the experiment (10 and 11).  Since we are not interested in the latter, and should keep the data on patient 4 regardless of metadata, we do a left-join of the data of the experiment, and the metadata.

Since gender and condition are categorical data, we set the type appropriately.

How many patients involved in our experiment are male, and how many female?

Which patients had a temperature higher than $39.5^{\circ}$, and when?  Let's also display the gender.

Suspiciously many males, let's see how many males versus females had a temperature (at any point).

## Gender differences?

There might be a gender influence here. Let's split our data set into male and female patients, again as a time series with the timestamp as index, but now with a multi-level column for gender and patient ID.

Note that now the data for patient 4 is missing, since the gender is unknown.

Let's compute the average temperature for male and female patients, compare.

These are two `Series`, we can concatenate these into a single `DataFrame`, en set the column names.

Let's make a plot of both average temperatures.

Looks like we are on to something, pity this is not real data.

## Conditions

However, we didn't take the patients' condition into account yet. Let's check how many patients have a specific condition.

What is the distribution of the condition with respect to the patients's gender?

Create a bar plot of this.

## From numbers to categories

Let's create an extra column in the `DataFrame` that is categorical, and represents the status of the patients in terms of fever.

A similar query to the one we did before can now be done on the `status` attribute.

The patients with high fever are given by: